# <font color=green>APR - Travaux Pratiques n°6.</font>

> Ce TP concerne la programmation CUDA. Les mêmes commentaires que ceux des derniers TP s’appliquent ici aussi. 
Dans cette séance, l’objectif est de pratiquer la programmation CUDA, autour des patrons en temps constants (sur machine PRAM). Le premier exercice est très proche du troisième exemple du cours, et ne devrait pas poser de difficulté. Le second exercice est un autre exemple de MAP qui vous semblera un peu bizarre voire étrange, mais sera ensuite utilisé dans les deux exercices suivants qui consistent à déplacer des morceaux d’images. Le dernier exercice, légèrement plus difficile, consistera à utiliser les contraintes matérielles d’un GPU au mieux, c’est-à-dire ici le principe d’accès coalescents à la mémoire globale.
>
> **<font color=pink>N'oubliez d'exécuter les deux premières cellules de code afin d'installer l'extension CUDA et de vérifier son bon fonctionnement.</font>**

## <font color=green>Installation du sous-sytème</font>

In [ ]:
# vérifions l'installation du SDK Cuda ...
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# Installons l'extension CUDA (n'hésitez par à aller sur la page GitHub ...)
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git &> /dev/null
%load_ext nvcc_plugin
# Installons gdown pour charger fichier depuis Google Drive
!pip install gdown &> /dev/null
# Installons g++-8
!sudo apt install g++-8 &> /dev/null
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-7 700 --slave /usr/bin/g++ g++ /usr/bin/g++-7
!sudo update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-8 800 --slave /usr/bin/g++ g++ /usr/bin/g++-8
# importation Python pour charger/afficher des images
from google.colab.patches import cv2_imshow
import cv2
def afficher(file, width):
  img = cv2.imread(file)
  height = int(img.shape[0] * width / float(img.shape[1]))
  resized = cv2.resize(img, (width, height), interpolation = cv2.INTER_AREA) 
  cv2_imshow(resized)

---
# <font color=green>Test</font>
---

>La cellule suivante contient un programme très simple, permettant de valider l'installation de l'extension CUDA.
>
>Si le programme ne fonctionne pas, peut-être est-ce parce que vous n'avez pas choisi le mode d'exécution "GPU" ? 
>
>Pour cela, dans le menu **<font color=pink>Exécution -> Modifier le type d'exécution</font>**, l'onglet **<font color=pink>Accélération matérielle</font>**, choisissez **<font color=pink>GPU</font>**.

In [ ]:
%%cu
#include <iostream>
__global__ void kernel(int *a) {
    a[threadIdx.x] = threadIdx.x;
}
int main() {
    constexpr int SIZE = 8;
    int*d_a;
    cudaMalloc(&d_a, sizeof(int)*SIZE);
    kernel<<<1,SIZE>>>(d_a);
    int h_a[SIZE];
    cudaMemcpy(h_a, d_a, sizeof(int)*SIZE, cudaMemcpyDeviceToHost);
    for(int i=0; i<SIZE; ++i) {
        if(h_a[i] != i) {
            std::cerr<<"CUDA DOES NOT WORK!\n";
            exit(-1);
        }
    }
    std::cerr<<"CUDA WORKS FINE!\n";
    return 0;
}

---
# <font color=green>TP</font>
> L'installation s'est bien déroulée ? Le test est concluant ? Parfait, maintenant au travail !
>
> En premier, il faut charger le TP6 depuis le drive Google ... Vous pouvez charger ce fichier (*i.e.* le premier, le second contient des images) sur votre ordinateur pour étudiez les interfaces, bien que la plupart soient dans le cours ...


In [ ]:
# Chargeons le TP6
!gdown https://drive.google.com/uc?id=1UXBWVkJR_pUJJDkuAa_YIPYb08oPIaTI
!gdown https://drive.google.com/uc?id=1FrHh5Pr2KlirwH6NBOrkfl-COSa9biTq
!unzip -oqq TP6.zip
!unzip -oqq Images-TP6.zip


>
> Le code du TP est dans le répertoire TP6. Vous pouvez le vérifier dans une cellule en tapant " !ls TP6" par exemple ...
>
> Nous démarrons avec l'exercice 1. 
---
## <font color=green>Exercice 1</font>
>
> **L’objectif de la fonction à écrire est de calculer la somme de deux tableaux, ici d’entiers (mais utilisez une version générique pour réutilisation future). La différence avec la version du cours est qu’ici la taille des tableaux n’est pas toujours un multiple du nombre de threads par bloc. Conséquence : il faut vérifier la pertinence de l’écriture dans le noyau …**
>
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP6/student/exo1/student.cu 
#include <iostream>
#include <exo1/student.h>
#include <OPP_cuda.cuh>

namespace 
{
	template<typename T>
	struct Plus 
	{
		__device__
		T operator()(const T& a, const T&b) const 
		{
			return a + b;
		}
	};

  // TODO: add your kernel here
	template<typename T, typename Functor>
	__global__
	void kernel(T const*const a, T const*const b, T *const c, const size_t size, 
	            Functor op)
 	{
		 	const size_t tid = threadIdx.x + blockIdx.x * blockDim.x;
			if( tid < size ) {
					c[tid] = op(a[tid], b[tid]);
			}
	}
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

// Attention : ici la taille des vecteurs n'est pas toujours un multiple du nombre de threads !
// Il faut donc corriger l'exemple du cours ...
void StudentWorkImpl::run_binary_map(
	OPP::CUDA::DeviceBuffer<int>& dev_a,
	OPP::CUDA::DeviceBuffer<int>& dev_b,
	OPP::CUDA::DeviceBuffer<int>& dev_result
) {
	// TODO
	// pour i de 0 à dev_a.getNbElements()-1 faire :
	//   dev_result[i] = dev_a[i] + dev_b[i];
	const auto nbElements = dev_a.getNbElements();
	dim3 threads(1024);
	dim3 blocs((nbElements + threads.x - 1) / threads.x);
	kernel<int, Plus<int>><<<blocs, threads>>>(
			dev_a.getDevicePointer(),
			dev_b.getDevicePointer(),
			dev_result.getDevicePointer(),
			nbElements,
			Plus<int>()
	);
}


In [ ]:
!ls TP6/utils
!cat TP6/utils/OPP/OPP_cuda_buffer.cuh

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP6 ; sh ./build.sh exo1

> ### <font color=green>Exécution</font>
> Exécutez la cellule suivante pour exécuter le code ...
>
> Pour le rapport, jouez avec la taille (pour les statistiques, cela signifie prendre des tailles importantes ...). 

In [ ]:
!./TP6/linux/exo1 -s=1000000

## <font color=green>Exercice 2</font>

> **La fonction à écrire reçoit deux images sous la forme de deux tableaux contenant les pixels (`uchar3`, au format R, G, B). Chaque tableau est organisé ligne par ligne. Votre fonction doit copier la première image dans la seconde, avec un effet vignette (cf. page 3). Cet effet consiste à ajouter un bord à chacun des $3\times3$ blocs de l’image. La couleur du bord est le troisième paramètre de la fonction à écrire. La largeur (nombre de pixels) du bord est le dernier paramètre.**
>
> **<font color=pink>NB : pensez parallèle ! Un algorithme séquentiel est inadaptable ...</font>**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Shift-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%cuda --name ../TP6/student/exo2/student.cu
#include <iostream>
#include <exo2/student.h>
#include <OPP_cuda.cuh>

namespace 
{
	// TODO
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_thumbnail(
	OPP::CUDA::DeviceBuffer<uchar3>& dev_inputImage,
	OPP::CUDA::DeviceBuffer<uchar3>& dev_outputImage,
	const uchar3 borderColor,
	const unsigned borderSize,
	const unsigned imageWidth, 
	const unsigned imageHeight
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP6 ; sh ./build.sh exo2

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!./TP6/linux/exo2 -i=./Images/Flower_600x450.ppm -b=3
# display result
afficher(file="Images/Flower_600x450_thumbnail.ppm", width = 600) 
# width = mettez une largeur en fonction de votre bande passante Internet 

In [ ]:
# launch student work
!./TP6/linux/exo2 -i=./Images/Raffael_012.ppm -b=3
# display result
afficher("Images/Raffael_012_thumbnail.ppm", 600)

In [ ]:
# launch student work
!./TP6/linux/exo2 -i=./Images/asphalt-highway.ppm -b=3
# display result
afficher("Images/asphalt-highway_thumbnail.ppm", 800)

## <font color=green>Exercice 3</font>

> **Votre fonction reçoit deux images (même principe que l’exercice 2), et doit effectuer une permutation par bloc pour transformer la première en la seconde (cf. page 4 du sujet PDF). La taille des blocs est le troisième paramètre de la fonction. Testez cela sur des images avec vignettes ;-)**
>
> **Le patron à appliquer ici est un GATHER.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Ctrl-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**


In [ ]:
%%cuda --name ../TP6/student/exo3/student.cu
#include <iostream>
#include <exo3/student.h>
#include <OPP_cuda.cuh>

namespace 
{
	// TODO
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_thumbnail_gather(
	OPP::CUDA::DeviceBuffer<uchar3>& dev_inputImage,
	OPP::CUDA::DeviceBuffer<uchar3>& dev_outputImage,
	OPP::CUDA::DeviceBuffer<uchar2>& dev_map,
	const unsigned imageWidth, 
	const unsigned imageHeight
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP6 ; sh ./build.sh exo3

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!./TP6/linux/exo3 -i=./Images/Flower_600x450_thumbnail.ppm
# display result
afficher(file="Images/Flower_600x450_thumbnail_gather.ppm", width = 600)

In [ ]:
# launch student work
!./TP6/linux/exo3 -i=./Images/Raffael_012_thumbnail.ppm
# display result
afficher("Images/Raffael_012_thumbnail_gather.ppm", 600)

In [ ]:
# launch student work
!./TP6/linux/exo3 -i=./Images/asphalt-highway_thumbnail.ppm -b=3
# display result
afficher("Images/asphalt-highway_thumbnail_gather.ppm", 800)

## <font color=green>Exercice 4</font>

> **Votre fonction reçoit deux images (même principe que l’exercice 2), et doit effectuer une permutation par bloc pour transformer la première en la seconde (cf. page 4). La taille des blocs est le troisième paramètre de la fonction.**
>
> **Le patron à appliquer ici est un SCATTER.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Ctrl-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP6/student/exo4/student.cu
#include <iostream>
#include <exo4/student.h>
#include <OPP_cuda.cuh>

namespace 
{
	// TODO
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_thumbnail_scatter(
	OPP::CUDA::DeviceBuffer<uchar3>& dev_inputImage,
	OPP::CUDA::DeviceBuffer<uchar3>& dev_outputImage,
	OPP::CUDA::DeviceBuffer<uchar2>& dev_map,
	const unsigned imageWidth, 
	const unsigned imageHeight
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP6 ; sh ./build.sh exo4

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...

In [ ]:
# launch student work
!./TP6/linux/exo4 -i=./Images/Flower_600x450_thumbnail.ppm
# display result
afficher(file = "Images/Flower_600x450_thumbnail_scatter.ppm", width = 600)

In [ ]:
# launch student work
!./TP6/linux/exo4 -i=./Images/Raffael_012_thumbnail.ppm
# display result
afficher("Images/Raffael_012_thumbnail_scatter.ppm", 600)

In [ ]:
# launch student work
!./TP6/linux/exo4 -i=./Images/asphalt-highway_thumbnail.ppm -b=3
# display result
afficher("Images/asphalt-highway_thumbnail_scatter.ppm", 800)

## <font color=green>Exercice 5</font>

> **Dans ce dernier exercice, il vous est demandé de tenir compte du mode d’accès à la mémoire globale depuis chaque warp (coalescence). L’idée est de filtrer une image (cf. page 5). Le filtre est une fonction fournie sous la forme d’un tableau de coefficients à appliquer pour calculer la valeur de chaque pixel de l’image produite. Il s’applique sur les pixels du voisinage du pixel de même position dans l’image source. Le filtre s’applique sur une grille carrée des voisins du pixel à filtrer, avec une taille impaire (e.g. $3\times3$, $5\times5$, $7\times7$, $\ldots$).</font>**
> **Par exemple, si le filtre est de taille $3\times3$, alors le calcul du pixel de position $\left(x,y\right)$ sera par composante R, G et B :**
>
> $$D\left(x,y\right)=\sum_{i=0}^{2}\sum_{j=0}^{2}F\left(i,j\right)\times I\left(x-i-1,y-i-1\right),$$
>
> **avec $D$ l’image destination (celle à écrire), $I$ l’image source et $F$ la fonction à appliquer. Notez que les « -1 » dans les indices de l’images sources viennent de la taille du filtre (dans cet exemple). En général il faut utiliser `size/2`. Si `size` est 3, vous obtenez 1 ! Lorsque `size` vaut 5, le résultat est 2 ; pour `size=7` vous aurez 3, etc.**
>
> **Notez que les pixels « manquants » (proche du bord) sont obtenus en repliant l’image sur elle-même (symétrie axiale). Par exemple le pixel `(-1,2)` deviendra par symétrie le pixel `(0,2)`. Le pixel `(-1,-2)` sera le pixel `(0,1)`. Le pixel `(width+3,height)` sera le pixel `(width-4,height-1)` puisque les derniers indices par ligne et colonne sont respectivement `width-1` et `height-1`.**
>
> **Expérimentez différents schémas d’accès/répartition des calculs pour définir la taille de la grille correctement.**
>
> ### <font color=green>Partie étudiante</font>
>
> La partie ci-dessous est pour vous. Répondez à l'exercice dans la cellule suivante. 
>
> Pour sauvegarder, n'oubliez pas de terminer par "Ctrl-Entrée" ... 
>
> **<font color=pink>Attention : ne touchez pas à la première ligne !</font>**

In [ ]:
%%cuda --name ../TP6/student/exo5/student.cu
include <iostream>
#include <exo5/student.h>
#include <OPP_cuda.cuh>

namespace 
{
	// Vous utiliserez ici les types uchar3 et float3 (internet : CUDA uchar3)
	// Addition de deux "float3"
	__device__ 
	float3 operator+(const float3 &a, const float3 &b) 
	{
		return make_float3(a.x+b.x, a.y+b.y, a.z+b.z);
	}

	// TODO
}

bool StudentWorkImpl::isImplemented() const {
	return true;
}

void StudentWorkImpl::run_filter(
	OPP::CUDA::DeviceBuffer<uchar3>& dev_inputImage,
	OPP::CUDA::DeviceBuffer<uchar3>& dev_outputImage,
	OPP::CUDA::DeviceBuffer<float>& dev_filter,
	const unsigned imageWidth, 
	const unsigned imageHeight,
	const unsigned filterWidth
) {
	// TODO
}

> ### <font color=green>Compilation</font>
> Exécutez la cellule suivante pour compiler le code ...

In [ ]:
!cd TP6 ; sh ./build.sh exo5

> ### <font color=green>Exécution</font>
> Exécutez les trois cellules suivantes pour exécuter le code (avec les images pré-chargées) ...
>
> Pour le rapport, jouez avec la taille (pour les statistiques). 

In [ ]:
# launch student work
!./TP6/linux/exo5 -i=./Images/Flower_600x450.ppm -f=11
# display result
afficher("Images/Flower_600x450_filtered.ppm", 600)

In [ ]:
# launch student work
!./TP6/linux/exo5 -i=./Images/Raffael_012.ppm -f=15
# display result
afficher("Images/Raffael_012_filtered.ppm", 600)

In [ ]:
# launch student work
!./TP6/linux/exo5 -i=./Images/asphalt-highway.ppm -f=63
# display result
afficher("Images/asphalt-highway_filtered.ppm", 800)

# <font color=green>That's all, folks!</font>